In [2]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
#from cftime import datetime 
from datetime import datetime, timedelta
#import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
from dask.distributed import LocalCluster, Client
import dask.array as da
import dask.dataframe as dd
import dask
from dask import delayed
from dask.diagnostics import ProgressBar  # Import the ProgressBar class
import calendar
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_2372439/2706121075.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [10]:
################################################################################
# Open WACCM dataset to define bins
################################################################################


# Define the relative path to the symbolic link
symbolic_link = 'Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport'

# Get the full path to the symbolic link
link_path = os.path.expanduser(os.path.expandvars(os.path.normpath(symbolic_link)))

# Join the symbolic link with the specific file you want to open
file_name = 'ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-01-01-00000.nc'
file_path = os.path.join(link_path, file_name)

# Open the file
wds = xr.open_dataset(file_path)
#wds

In [11]:
wds_lat = wds['lat']
wds_lon = wds['lon']
wds_lev = wds['lev']
wds_ilev = wds['ilev']
wds_geopH = wds.variables['Z3'] / 1000 #m-> km
wds_Zavg = wds_geopH.mean(('time','lat', 'lon'))
wds_time = wds['time']

# Slice arrays (lev, altitude) between chosen range (~80-130km)
lev_sl_idx_min = 42
lev_sl_idx_max = 66

# Slice arrays (lev & alt) between chosen range & print
lev_sl = wds_lev[lev_sl_idx_min:lev_sl_idx_max+1]
Zavg_sl = wds_Zavg[lev_sl_idx_min:lev_sl_idx_max+1]


print('    Array lev = ' + str("%.1e" % wds_lev[lev_sl_idx_max] ) + 'hPa : ' + str("%.1e" % wds_lev[lev_sl_idx_min] ) + 'hPa'
           + ' (approx ' + str("%.0f" % wds_Zavg[lev_sl_idx_max]) + 'km : ' + str("%.0f" % wds_Zavg[lev_sl_idx_min] ) + 'km' + ')'   )


    Array lev = 6.0e-03hPa : 1.5e-05hPa (approx 81km : 126km)


In [12]:
################################################################################
# Lat bins
################################################################################

lat_values = wds_lat.values

lat_bin_edges = lat_values
lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
#lat_bin_widths = np.diff(lat_bin_edges)
#lat_average_bin_width = np.mean(lat_bin_widths)
lat_num_bins = len(lat_bin_midpoints)


# ##Print the results
print("Lat Bin Edges (Lower and Upper):")
for i in range(len(lat_bin_edges) - 1):
    print(f"Bin {i + 1}: [{lat_bin_edges[i]}, {lat_bin_edges[i + 1]}]")

# print("\nLat Bin Midpoints:")
# for i in range(len(lat_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {lat_bin_midpoints[i]}")

# print("\nLat Bin Widths:", lat_bin_widths)
# print(f"Lat Average Bin Width: {lat_average_bin_width:.6f} degrees")
# print(f"Lat Number of Bins: {lat_num_bins}")


# # Lat Average Bin Width: 1.894737 degrees
# # Lat Number of Bins: 95

Lat Bin Edges (Lower and Upper):
Bin 1: [-90.0, -88.10526315789474]
Bin 2: [-88.10526315789474, -86.21052631578948]
Bin 3: [-86.21052631578948, -84.3157894736842]
Bin 4: [-84.3157894736842, -82.42105263157895]
Bin 5: [-82.42105263157895, -80.52631578947368]
Bin 6: [-80.52631578947368, -78.63157894736842]
Bin 7: [-78.63157894736842, -76.73684210526316]
Bin 8: [-76.73684210526316, -74.84210526315789]
Bin 9: [-74.84210526315789, -72.94736842105263]
Bin 10: [-72.94736842105263, -71.05263157894737]
Bin 11: [-71.05263157894737, -69.15789473684211]
Bin 12: [-69.15789473684211, -67.26315789473685]
Bin 13: [-67.26315789473685, -65.36842105263158]
Bin 14: [-65.36842105263158, -63.473684210526315]
Bin 15: [-63.473684210526315, -61.578947368421055]
Bin 16: [-61.578947368421055, -59.684210526315795]
Bin 17: [-59.684210526315795, -57.78947368421053]
Bin 18: [-57.78947368421053, -55.89473684210527]
Bin 19: [-55.89473684210527, -54.0]
Bin 20: [-54.0, -52.10526315789474]
Bin 21: [-52.10526315789474, -5

In [13]:
################################################################################
# Lon bins
################################################################################

lon_upper_edge_last_bin = 360.0
lon_bin_edges = np.append(wds_lon, lon_upper_edge_last_bin)

lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2
#lon_bin_widths = np.diff(lon_bin_edges)
#lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)


## Print the results
print("Lon Bin Edges (Lower and Upper):")
for i in range(len(lon_bin_edges) - 1):
    print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

# print("\nLon Bin Midpoints:")
# for i in range(len(lon_bin_midpoints)):
#     print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")


#Average Lon Bin Width: 2.50000000 degrees
#Number of Lon Bins: 144

Lon Bin Edges (Lower and Upper):
Bin 1: [0.0, 2.5]
Bin 2: [2.5, 5.0]
Bin 3: [5.0, 7.5]
Bin 4: [7.5, 10.0]
Bin 5: [10.0, 12.5]
Bin 6: [12.5, 15.0]
Bin 7: [15.0, 17.5]
Bin 8: [17.5, 20.0]
Bin 9: [20.0, 22.5]
Bin 10: [22.5, 25.0]
Bin 11: [25.0, 27.5]
Bin 12: [27.5, 30.0]
Bin 13: [30.0, 32.5]
Bin 14: [32.5, 35.0]
Bin 15: [35.0, 37.5]
Bin 16: [37.5, 40.0]
Bin 17: [40.0, 42.5]
Bin 18: [42.5, 45.0]
Bin 19: [45.0, 47.5]
Bin 20: [47.5, 50.0]
Bin 21: [50.0, 52.5]
Bin 22: [52.5, 55.0]
Bin 23: [55.0, 57.5]
Bin 24: [57.5, 60.0]
Bin 25: [60.0, 62.5]
Bin 26: [62.5, 65.0]
Bin 27: [65.0, 67.5]
Bin 28: [67.5, 70.0]
Bin 29: [70.0, 72.5]
Bin 30: [72.5, 75.0]
Bin 31: [75.0, 77.5]
Bin 32: [77.5, 80.0]
Bin 33: [80.0, 82.5]
Bin 34: [82.5, 85.0]
Bin 35: [85.0, 87.5]
Bin 36: [87.5, 90.0]
Bin 37: [90.0, 92.5]
Bin 38: [92.5, 95.0]
Bin 39: [95.0, 97.5]
Bin 40: [97.5, 100.0]
Bin 41: [100.0, 102.5]
Bin 42: [102.5, 105.0]
Bin 43: [105.0, 107.5]
Bin 44: [107.5, 110.0]
Bin 45: [110.0, 112.5]
Bin 46: [112.5, 115.0]
Bin 

In [14]:
################################################################################
# Zavg_sl bins - Note first bin is at top
################################################################################

# Convert xarray Variable to NumPy array
Zavg_sl_values = Zavg_sl.values

Zavg_sl_bin_widths = np.diff(Zavg_sl_values)
Zavg_sl_bin_edges = [Zavg_sl_values[0] - Zavg_sl_bin_widths[0] / 2] + [Zavg_sl_values[i] + Zavg_sl_bin_widths[i] / 2 for i in range(len(Zavg_sl_bin_widths))] + [Zavg_sl_values[-1] + Zavg_sl_bin_widths[-1] / 2]
Zavg_sl_bin_midpoints = [(Zavg_sl_bin_edges[i] + Zavg_sl_bin_edges[i + 1]) / 2 for i in range(len(Zavg_sl_bin_edges) - 1)]
#Zavg_sl_bin_widths = [Zavg_sl_bin_edges[i + 1] - Zavg_sl_bin_edges[i] for i in range(len(Zavg_sl_bin_edges) - 1)]
Zavg_sl_num_bins = len(Zavg_sl_bin_midpoints)


# # Print the bin edges and midpoints
# print("Zavg_sl Bin Edges (Lower and Upper):")
# for i in range(len(Zavg_sl_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{Zavg_sl_bin_edges[i]}, {Zavg_sl_bin_edges[i + 1]}")

# print("\nZavg_sl Bin Midpoints:")
# for i in range(len(Zavg_sl_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {Zavg_sl_bin_midpoints[i]}")

# # Print the bin widths
# print("\nZavg_sl Bin Widths:")
# for i in range(len(Zavg_sl_bin_widths)):
#     print(f"Bin {i + 1} Width: {Zavg_sl_bin_widths[i]}")

# print(f"Number of Zavg_sl_bin_widths Bins: {Zavg_sl_num_bins}")

#Number of Zavg_sl_bin_widths Bins: 25

In [15]:
LT_bin_edges = np.linspace(0, 24, 49)
LT_bin_edges[-1] = 24.0

# Calculate bin midpoints
LT_bin_midpoints = (LT_bin_edges[:-1] + LT_bin_edges[1:]) / 2

# Calculate the number of bins
LT_num_bins = len(LT_bin_edges) - 1

# Calculate bin widths
LT_bin_widths = np.diff(LT_bin_edges)

# Calculate the average bin width
average_bin_width = np.sum(LT_bin_widths) / LT_num_bins


##Print the results
print("LT Bin Edges (Lower and Upper):")
for i in range(len(LT_bin_edges) - 1):
    print(f"Bin {i + 1}: [{LT_bin_edges[i]}, {LT_bin_edges[i + 1]}]")

print("\nLT Bin Midpoints:")
for i in range(len(LT_bin_midpoints)):
    print(f"Bin {i + 1} Midpoint: {LT_bin_midpoints[i]}")

print("\nNumber of LT Bins:")
print(LT_num_bins)

# Print the average bin width
print("Average LT Bin Width:", average_bin_width)

LT Bin Edges (Lower and Upper):
Bin 1: [0.0, 0.5]
Bin 2: [0.5, 1.0]
Bin 3: [1.0, 1.5]
Bin 4: [1.5, 2.0]
Bin 5: [2.0, 2.5]
Bin 6: [2.5, 3.0]
Bin 7: [3.0, 3.5]
Bin 8: [3.5, 4.0]
Bin 9: [4.0, 4.5]
Bin 10: [4.5, 5.0]
Bin 11: [5.0, 5.5]
Bin 12: [5.5, 6.0]
Bin 13: [6.0, 6.5]
Bin 14: [6.5, 7.0]
Bin 15: [7.0, 7.5]
Bin 16: [7.5, 8.0]
Bin 17: [8.0, 8.5]
Bin 18: [8.5, 9.0]
Bin 19: [9.0, 9.5]
Bin 20: [9.5, 10.0]
Bin 21: [10.0, 10.5]
Bin 22: [10.5, 11.0]
Bin 23: [11.0, 11.5]
Bin 24: [11.5, 12.0]
Bin 25: [12.0, 12.5]
Bin 26: [12.5, 13.0]
Bin 27: [13.0, 13.5]
Bin 28: [13.5, 14.0]
Bin 29: [14.0, 14.5]
Bin 30: [14.5, 15.0]
Bin 31: [15.0, 15.5]
Bin 32: [15.5, 16.0]
Bin 33: [16.0, 16.5]
Bin 34: [16.5, 17.0]
Bin 35: [17.0, 17.5]
Bin 36: [17.5, 18.0]
Bin 37: [18.0, 18.5]
Bin 38: [18.5, 19.0]
Bin 39: [19.0, 19.5]
Bin 40: [19.5, 20.0]
Bin 41: [20.0, 20.5]
Bin 42: [20.5, 21.0]
Bin 43: [21.0, 21.5]
Bin 44: [21.5, 22.0]
Bin 45: [22.0, 22.5]
Bin 46: [22.5, 23.0]
Bin 47: [23.0, 23.5]
Bin 48: [23.5, 24.0]

LT Bin 

In [4]:
### Raw satellite data ###
filepath = 'Nc_Files/s4max_files/1_s4max.nc'

### Processed time data into datetime format ###
#new_filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### My processed data - 80-130km only ###
#result_filepath = 'Nc_Files/s4max_files/mod_s4max_80-130km.nc'

ds = xr.open_dataset(filepath)
ds

<xarray.Dataset>
Dimensions:   (nfile: 9266536)
Dimensions without coordinates: nfile
Data variables:
    Ut_time   (nfile) float64 ...
    altitude  (nfile) float64 ...
    DOY       (nfile) float64 ...
    lat       (nfile) float64 ...
    LT        (nfile) float64 ...
    lon       (nfile) float64 ...
    mlat      (nfile) float64 ...
    mlon      (nfile) float64 ...
    mlt       (nfile) float64 ...
    s4max     (nfile) float64 ...
    year      (nfile) float64 ...

In [9]:
# ds_lat = ds['lat']
# ds_lon = ds['lon']
# ds_alt = ds['altitude']
# ds_s4max = ds['s4max']

# ds_time = ds['time']
# ds_year = ds['year']

In [ ]:
####################################################################################

In [9]:
# TO DELETE ONCE I HAVE A WORKING VERSION OF THIS CODE BELOW #
#First attempt with loops but data array too big

# timesteps_2006 = 3458
# timesteps_2007 = 1481291
# timesteps_2008 = 1532493
# timesteps_2009 = 1659634
# timesteps_2010 = 1293973
# timesteps_2011 = 1100196
# timesteps_2012 = 963055
# timesteps_2013 = 1156574
# timesteps_2014 = 75862



# Create a dictionary to store s4max data arrays for each year
s4max_data_dict = {}


years = range(2006, 2015)

for year in years:
    year_mask = ds['time'].dt.year == year

    # Apply the mask using .where()
    year_data = ds.where(year_mask, drop=True)
    # print(year_data)
    year_time_steps = len(year_data["time"])
    print(f'Year {year}: time steps = {year_time_steps}')

    # Now select other variables using the same mask
    year_lat = ds['lat'].sel(nfile=year_data['nfile'])
    year_lon = ds['lon'].sel(nfile=year_data['nfile'])
    year_alt = ds['altitude_var'].sel(nfile=year_data['nfile'])
    year_s4max = ds['s4max'].sel(nfile=year_data['nfile'])
    year_time = ds['time'].sel(nfile=year_data['nfile'])

    
    # Loop through your data to categorize s4max
    s4max_data = np.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, year_time_steps))
    s4max_data.fill(np.nan)
    
    # Loop through your data to categorize s4max
    for i in tqdm(range(year_time_steps), desc=f'Year {year}'):
        current_altitude = year_alt.values[i]
        
        current_lat = year_lat.values[i]
        current_lon = year_lon.values[i]
        current_s4max = year_s4max.values[i]
        current_time = year_time.values[i]

        # Check if the current_altitude is within the range of your defined bins
        if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[-2]:

            #print('current_lat = ' + str(current_lat))
            # print('current_lon = ' + str(current_lon))
            #print('current_altitude = ' + str(current_altitude))
            # print('current_s4max = ' + str(current_s4max))
            # print('current_time = ' + str(current_time))


            # Find the bin indices for latitude, longitude, and altitude
            lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
            lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
            altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges) - 1
            #print('lat_bin_index = ' + str(lat_bin_index))
            #print('altitude_bin_index = ' + str(altitude_bin_index))


            # Update the corresponding entry in the s4max_data array
            s4max_data[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max

        # Store s4max data for the current year in the dictionary with a variable name
        s4max_data_dict[f's4max_{year}'] = s4max_data

    # # Access s4max data for a specific year, e.g., s4max data for 2006
    # s4max_2006 = s4max_data_dict['s4max_2006']
    # s4max_2006   


Year 2006: time steps = 3458


NameError: name 'Zavg_sl_num_bins' is not defined

In [10]:
# TO DELETE ONCE I HAVE A WORKING VERSION OF THIS CODE BELOW #
#Second attempt with dask but data not being added to final s4max array properly (see cell below where time dim = 0)

# timesteps_2006 = 3458
# timesteps_2007 = 1481291
# timesteps_2008 = 1532493
# timesteps_2009 = 1659634
# timesteps_2010 = 1293973
# timesteps_2011 = 1100196
# timesteps_2012 = 963055
# timesteps_2013 = 1156574
# timesteps_2014 = 75862


import calendar

# Create a dictionary to store s4max data arrays for each year
s4max_data_dict = {}

years = range(2006, 2015)

for year in years:
    year_mask = ds_time.dt.year == year

    # Apply the mask using .where()
    year_data = ds.where(year_mask, drop=True)
    # print(year_data)
    year_time_steps = len(year_data["time"])
    print(f'Year {year}: time steps = {year_time_steps}')

    # Now select other variables using the same mask
    year_lat = ds['lat'].sel(nfile=year_data['nfile'])
    year_lon = ds['lon'].sel(nfile=year_data['nfile'])
    year_alt = ds['altitude_var'].sel(nfile=year_data['nfile'])
    year_s4max = ds['s4max'].sel(nfile=year_data['nfile'])
    year_time = ds['time'].sel(nfile=year_data['nfile'])

    for month in range(1, 13):
        month_name = calendar.month_name[month]

        # Create a mask for the current month
        month_mask = year_data["time"].dt.month == month
        month_data = year_data.where(month_mask, drop=True)
        month_time_steps = len(month_data["time"])


        # # Create a NumPy array filled with NaN
        # nan_array = np.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, month_time_steps))
        # nan_array.fill(np.nan)
        # # Convert the NumPy array to a Dask array
        # s4max_data = da.from_array(nan_array, chunks=(-1, -1, -1, -1))
        
        s4max_data = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, month_time_steps), chunks=(-1, -1, -1, -1))
            
        
        for i in tqdm(range(month_time_steps), desc=f'Year {year}, Month {month_name}'):
            current_altitude = month_data['altitude_var'].values[i]
            current_lat = month_data['lat'].values[i]
            current_lon = month_data['lon'].values[i]
            current_s4max = month_data['s4max'].values[i]
            current_time = month_data["time"].values[i]

            # Check if the current_altitude is within the range of your defined bins
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[-2]:
                # Find the bin indices for latitude, longitude, and altitude
                lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges) - 1

                # Update the corresponding entry in the s4max_data array
                s4max_data[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max

        # Store s4max data for the current year and month in the dictionary with a variable name
        s4max_data_dict[f's4max_{year}_{month_name}'] = s4max_data


Year 2006: time steps = 3458
Year 2007: time steps = 1481291
Year 2008: time steps = 1532493
Year 2009: time steps = 1659634
Year 2010: time steps = 1293973
Year 2011: time steps = 1100196
Year 2012: time steps = 963055
Year 2013: time steps = 1156574
Year 2014: time steps = 75862


In [44]:
year_id = '2006'
month_id = 'December'

s4max_cur = s4max_data_dict[f's4max_{year}_{month_name}']
#s4max_cur 
print(s4max_cur.shape) 

(25, 95, 144, 0)


In [11]:
# TO DELETE ONCE I HAVE A WORKING VERSION OF THIS CODE BELOW #
#Third attempt with dask - issues with masking. Year masking looks ok but month masking is losing data in time dim

import calendar

# Create a dictionary to store s4max data arrays for each year
s4max_data_dict = {}

years = range(2006, 2015)

for year in years:
    for month in range(1, 13):
        year_mask = ds_time.dt.year == year
        
        month_mask = ds_time.dt.month == month

        # Combine the year and month masks
        combined_mask = year_mask & month_mask

        # Apply the combined mask using .where()
        masked_data = ds.where(year_mask, drop=True)
        
        masked_time_steps = len(masked_data["time"])
        print(masked_time_steps)
        
        masked_name = calendar.month_name[month]
       
        # Select other variables for the current month
        masked_lat = masked_data['lat']
        masked_lon = masked_data['lon']
        masked_alt = masked_data['altitude_var']
        masked_s4max = masked_data['s4max']
        masked_time = masked_data['time']
        print(masked_time.values)

        
#         s4max_data = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, month_time_steps), chunks=(-1, -1, -1, -1))
            
        
#         for i in tqdm(range(month_time_steps), desc=f'Year {year}, Month {month_name}'):
#             current_altitude = month_data['altitude_var'].values[i]
#             current_lat = month_data['lat'].values[i]
#             current_lon = month_data['lon'].values[i]
#             current_s4max = month_data['s4max'].values[i]
#             current_time = month_data["time"].values[i]

#             # Check if the current_altitude is within the range of your defined bins
#             if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[-2]:
#                 # Find the bin indices for latitude, longitude, and altitude
#                 lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
#                 lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
#                 altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges) - 1

#                 # Update the corresponding entry in the s4max_data array
#                 s4max_data[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max

#         # Store s4max data for the current year and month in the dictionary with a variable name
#         s4max_data_dict[f's4max_{year}_{month_name}'] = s4max_data

3458
['2006-12-28T21:56:00.000000000' '2006-12-28T12:07:00.000000000'
 '2006-12-28T10:09:00.000000000' ... '2006-12-31T17:59:00.000000000'
 '2006-12-31T08:03:00.000000000' '2006-12-31T02:10:00.000000000']
3458
['2006-12-28T21:56:00.000000000' '2006-12-28T12:07:00.000000000'
 '2006-12-28T10:09:00.000000000' ... '2006-12-31T17:59:00.000000000'
 '2006-12-31T08:03:00.000000000' '2006-12-31T02:10:00.000000000']
3458
['2006-12-28T21:56:00.000000000' '2006-12-28T12:07:00.000000000'
 '2006-12-28T10:09:00.000000000' ... '2006-12-31T17:59:00.000000000'
 '2006-12-31T08:03:00.000000000' '2006-12-31T02:10:00.000000000']


KeyboardInterrupt: 

In [31]:
# TO DELETE ONCE I HAVE A WORKING VERSION OF THIS CODE BELOW #

#Fourth attempt with dask - issues with masking fixed. Think it was just a mistake in the altitude selection criteria. Anyhow it has changed a bit and seems to work. 
#However, it takes forever to do... so below I am trying a way to go through month by month and make nc files


# Create a dictionary to store s4max data arrays for each year
s4max_data_dict = {}

years = range(2006, 2015)

for year in years:
    year_mask = ds_time.dt.year == year
    year_data = ds.where(year_mask, drop=True)

    # year_time_steps = len(year_data["time"])
    # print(f'Year {year}: time steps = {year_time_steps}')

    # # Now select other variables using the same mask
    # year_lat = ds['lat'].sel(nfile=year_data['nfile'])
    # year_lon = ds['lon'].sel(nfile=year_data['nfile'])
    # year_alt = ds['altitude_var'].sel(nfile=year_data['nfile'])
    # year_s4max = ds['s4max'].sel(nfile=year_data['nfile'])
    # year_time = ds['time'].sel(nfile=year_data['nfile'])

    for month in range(1, 13):
        month_name = calendar.month_name[month]
        
        # Create a mask for the current month
        month_mask = year_data["time"].dt.month == month
        month_data = year_data.where(month_mask, drop=True)  

        month_time_steps = len(month_data["time"])
        # print(f'Year {year} Month {month}: time steps = {month_time_steps}')
        
        # # Now select other variables using the same mask for the selected month
        # month_lat = ds['lat'].sel(nfile=month_data['nfile'])
        # month_lon = ds['lon'].sel(nfile=month_data['nfile'])
        # month_alt = ds['altitude_var'].sel(nfile=month_data['nfile'])
        # month_s4max = ds['s4max'].sel(nfile=month_data['nfile'])
        # month_time = ds['time'].sel(nfile=month_data['nfile'])

        s4max_data = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, month_time_steps), chunks=(-1, -1, -1, -1))
            
        
        for i in tqdm(range(month_time_steps), desc=f'Year {year}, Month {month_name}'):
            current_altitude = month_data['altitude_var'].values[i]
            current_lat = month_data['lat'].values[i]
            current_lon = month_data['lon'].values[i]
            current_s4max = month_data['s4max'].values[i]
            current_time = month_data["time"].values[i]

            # Check if the current_altitude is within the range of your defined bins
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:

                lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges) - 1

                # Update the corresponding entry in the s4max_data array
                s4max_data[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max

        # Store s4max data for the current year and month in the dictionary with a variable name
        s4max_data_dict[f's4max_{year}_{month_name}'] = s4max_data

Year 2006, Month January: 0it [00:00, ?it/s]
Year 2006, Month February: 0it [00:00, ?it/s]
Year 2006, Month March: 0it [00:00, ?it/s]
Year 2006, Month April: 0it [00:00, ?it/s]
Year 2006, Month May: 0it [00:00, ?it/s]
Year 2006, Month June: 0it [00:00, ?it/s]
Year 2006, Month July: 0it [00:00, ?it/s]
Year 2006, Month August: 0it [00:00, ?it/s]
Year 2006, Month September: 0it [00:00, ?it/s]
Year 2006, Month October: 0it [00:00, ?it/s]
Year 2006, Month November: 0it [00:00, ?it/s]
Year 2007, Month May:  74%|██████████████████████████████████████████████████████████████████████████████████████████                                | 121919/165236 [24:29<08:42, 82.95it/s]


KeyboardInterrupt: 

In [ ]:
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * Working code?!* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##


In [33]:
# TO DELETE ONCE I HAVE A WORKING VERSION OF THIS CODE BELOW #
#Fifth attempt with dask - can create the ds for each month OK but taking impossibly long to write to nc file


# Manually set the year you want to process
year = 2006  # Set the desired year
desired_months = [12]  # Set the desired months

year_mask = ds_time.dt.year == year
year_data = ds.where(year_mask, drop=True)

# # Define a directory to save the netCDF files
# output_directory = "./Nc_Files/s4max/"

# # Define a Dask function for writing data to NetCDF files
# @delayed
# def write_to_netcdf(data, output_file):
#     data.to_netcdf(output_file)

# # Create a list to store Dask tasks for writing data to NetCDF files
# write_tasks = []


for month in desired_months:
    month_name = calendar.month_name[month]

    # Create a mask for the current month
    month_mask = year_data["time"].dt.month == month
    month_data = year_data.where(month_mask, drop=True)

    month_time_steps = len(month_data["time"])

    s4max_lon = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, month_time_steps), chunks=(-1, -1, -1, -1))
    s4max_LT = da.empty((Zavg_sl_num_bins, lat_num_bins, LT_num_bins, month_time_steps), chunks=(-1, -1, -1, -1))


    for i in tqdm(range(month_time_steps), desc=f'Year {year}, Month {month_name}'):
        current_altitude = month_data['altitude_var'].values[i]
        current_lat = month_data['lat'].values[i]
        current_lon = month_data['lon'].values[i]
        current_s4max = month_data['s4max'].values[i]
        current_time = month_data["time"].values[i]
        current_LT = month_data["LT"].values[i]

        # Check if the current_altitude is within the range of your defined bins
        if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
            lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
            lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
            altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
            LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1

            # Update the corresponding entry in the s4max_data array
            s4max_lon[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max
            s4max_LT[altitude_bin_index, lat_bin_index, LT_bin_index, i] = current_s4max
    
    # Create a new xarray dataset for the s4max data
    s4max_lon_ds = xr.Dataset(
        data_vars={"s4max": (['altitude', 'latitude', 'longitude', 'time'], s4max_lon)},
        coords={
            "altitude": Zavg_sl_bin_edges[:-1],
            "latitude": lat_bin_edges[:-1],
            "longitude": lon_bin_edges[:-1],
            "time": month_data["time"].values,
        }
    )

    s4max_LT_ds = xr.Dataset(
        data_vars={"s4max": (['altitude', 'latitude', 'local_time', 'time'], s4max_LT)},
        coords={
            "altitude": Zavg_sl_bin_edges[:-1],
            "latitude": lat_bin_edges[:-1],
            "local_time": LT_bin_edges[:-1],
            "time": month_data["time"].values,
        }
    )


#     # Save the month's s4max data into a NetCDF file
#     output_file = f"{output_directory}s4max_{year}_{month_name}.nc"
#     #s4max_ds.to_netcdf(output_file)

#     # Delay the write operation using Dask
#     write_task = write_to_netcdf(s4max_ds, output_file)
#     write_tasks.append(write_task)  # Append the Dask write tasks


Year 2006, Month December: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3458/3458 [00:03<00:00, 913.87it/s]


In [ ]:
# # Enable the progress bar
# with ProgressBar():
#     # Execute the Dask tasks to perform the write operations
#     dask.compute(write_tasks)

# # Disable the progress bar after completion
# ProgressBar().unregister()

In [38]:
# Average over the time dimension
s4lonavg_lon_lat_alt = s4max_lon_ds['s4max'].mean(dim='time')

# Average over the altitude dimension
s4lonavg_lon_lat = s4lonavg_lon_lat_alt.mean(dim='altitude')
s4lonavg_lon_lat = s4lonavg_lon_lat.compute()

KeyboardInterrupt: 

In [ ]:
#
#                  THIS
#                   ONE
#
#


In [10]:
#Sixth attempt - same as 5 but processing day by day. 
# then calculating daily and dataset (monthly/3monthly) averages. Tried computing and accessing the data after but crashing.

# LOCAL TIME AND LON

start_time = datetime.now()
print("Start time: ", start_time)


# Manually set the year you want to process
year = 2006  # Set the desired year
desired_months = [12]#, 4]#, 5]  # Set the desired months

year_mask = ds["time"].dt.year == year
year_data = ds.where(year_mask, drop=True)

# Create an empty list to accumulate datasets
daily_lon_datasets = []
daily_LT_dataset1 = []
daily_LT_dataset2 = []

ssn_lon_dataset = []
ssn_LT_dataset1 = []
ssn_LT_dataset2 = []

# Calculate the desired averaging_period based on your logic
if len(desired_months) == 1:
    averaging_period = "Monthly"
elif len(desired_months) == 2:
    averaging_period = "Two-Month"
elif len(desired_months) == 3:
    averaging_period = "Three-Month"
else:
    averaging_period = "Unknown"  # You can specify another value for other cases


for month in desired_months:
    month_name = calendar.month_name[month]

    # Create a mask for the current month
    month_mask = year_data["time"].dt.month == month
    month_data = year_data.where(month_mask, drop=True)

    # Month data is filtered, now filter by day
    for day in range(1, calendar.monthrange(year, month)[1] + 1):
        day_mask = month_data["time"].dt.day == day
        day_data = month_data.where(day_mask, drop=True)
        day_time_steps = len(day_data["time"])
        
        s4max_lon = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, day_time_steps), chunks=(-1, -1, -1, -1))
        s4max_LT = da.empty((Zavg_sl_num_bins, lat_num_bins, LT_num_bins, day_time_steps), chunks=(-1, -1, -1, -1))
        
        # Create a timestamp using these values
        day_timestamp = pd.Timestamp(year=year, month=month, day=day)

        if day_time_steps > 0:
            for i in tqdm(range(day_time_steps), desc=f'Year {year}, Month {month_name}, Day {day}'):
                current_altitude = day_data['altitude_var'].values[i]
                current_lat = day_data['lat'].values[i]
                current_lon = day_data['lon'].values[i]
                current_s4max = day_data['s4max'].values[i]
                current_time = day_data["time"].values[i]
                current_LT = day_data["LT"].values[i]

                # Check if the current_altitude is within the range of your defined bins
                if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                    lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                    lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                    altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                    
                    # Calculate LT_bin_index and manually assign it to the last bin if it's equal to the upper bound
                    LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                    if current_LT == LT_bin_edges[-1]:
                        LT_bin_index = len(LT_bin_edges) - 2

                    # Update the corresponding entry in the s4max data array
                    s4max_lon[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max
                    #s4max_LT[altitude_bin_index, lat_bin_index, LT_bin_index, i] = current_s4max

            #//////////////////////////////////
            # Create a new xarray dataset for the s4max lon data. One day of data at all time steps for that day
            s4max_lon_ds = xr.Dataset(
                data_vars={"s4max": (['altitude', 'latitude', 'longitude', 'time'], s4max_lon)},
                coords={
                    "altitude": Zavg_sl_bin_edges[:-1],
                    "latitude": lat_bin_edges[:-1],
                    "longitude": lon_bin_edges[:-1],
                    "time": day_data["time"].values,
                }
            )
            
            # # Create a new xarray dataset for the s4max LT data. One day of data at all time steps for that day
            # s4max_LT_ds = xr.Dataset(
            #     data_vars={"s4max": (['altitude', 'latitude', 'local_time', 'time'], s4max_LT)},
            #     coords={
            #         "altitude": Zavg_sl_bin_edges[:-1],
            #         "latitude": lat_bin_edges[:-1],
            #         "local_time": LT_bin_edges[:-1],
            #         "time": day_data["time"].values,
            #     }
            # )
            # #//////////////////////////////////
            
            #--------------------------------
            # Average s4max_lon_ds over time dimension
            s4lonavg_alt_lat_lon = s4max_lon_ds['s4max'].mean(dim='time', keep_attrs=True) # daily avg 
            # Average s4lonavg_alt_lat_lon over alt dimension
            s4lonavg_lat_lon = s4lonavg_alt_lat_lon.mean(dim='altitude', keep_attrs=True) # *** daily avg over all alts ***
            # Now create a new DataArray with the desired variable name 's4max'
            s4lonavg_lat_lon = xr.Dataset(
                {
                    's4max': s4lonavg_lat_lon.expand_dims(time=[day_timestamp])
                },
                coords={
                    'latitude': s4lonavg_lat_lon['latitude'],
                    'longitude': s4lonavg_lat_lon['longitude'],
                    'time': [day_timestamp],
                }
            )   
            # #--------------------------------
            # # Average s4max_LT_ds over time dimension 
            # s4LTavg_alt_lat_LT = s4max_LT_ds['s4max'].mean(dim='time', keep_attrs=True) # *** daily avg ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat_LT = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat_LT.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat_LT['latitude'],
            #         'latitude': s4LTavg_alt_lat_LT['latitude'],
            #         'local_time': s4LTavg_alt_lat_LT['local_time'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
            # # Average s4LTavg_alt_lat_LT over LT dimension
            # s4LTavg_alt_lat = s4LTavg_alt_lat_LT.mean(dim='local_time', keep_attrs=True) # *** daily avg over all LTs ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat['altitude'],
            #         'latitude': s4LTavg_alt_lat['latitude'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
        

    
            # # Append daily datasets to the list
            daily_lon_datasets.append(s4lonavg_lat_lon)
            # daily_LT_dataset1.append(s4LTavg_alt_lat_LT)
            # daily_LT_dataset2.append(s4LTavg_alt_lat)
            #--------------------------------
            

ssn_lon_dataset = xr.concat(daily_lon_datasets, dim='time', data_vars='all')
# ssn_LT_dataset1 = xr.concat(daily_LT_dataset1, dim='time', data_vars='all')
# ssn_LT_dataset2 = xr.concat(daily_LT_dataset2, dim='time', data_vars='all')


ssn_s4lonavg_lat_lon_da = ssn_lon_dataset['s4max'].mean(dim='time', keep_attrs=True)
# ssn_s4LTavg_alt_lat_LT = ssn_LT_dataset1['s4max'].mean(dim='time', keep_attrs=True)
# ssn_s4LTavg_alt_lat = ssn_LT_dataset2['s4max'].mean(dim='time', keep_attrs=True)


# Now create a new DataArray with the desired variable name 's4max'
ssn_s4lonavg_lat_lon_ds = xr.Dataset(
    data_vars={"s4max": ssn_s4lonavg_lat_lon_da},
    attrs={'averaging_period': averaging_period}
)   



end_time = datetime.now()
time_elapsed = end_time - start_time
print("Time elapsed: ", time_elapsed)


Start time:  2023-10-31 13:31:41.764665


Year 2006, Month December, Day 31: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [00:00<00:00, 2641.40it/s]

Time elapsed:  0:00:03.122000


In [11]:
with ProgressBar():
    computed_data = ssn_s4lonavg_lat_lon_ds.compute()

#14m 42s for 3 days


[########################################] | 100% Completed | 14m 34s


In [12]:
#print(ssn_s4lonavg_lat_lon_ds['s4max'].shape) #95,144

s4max = computed_data['s4max']

In [18]:
ssn_s4lonavg_lat_lon_ds

<xarray.Dataset>
Dimensions:    (latitude: 95, longitude: 144)
Coordinates:
  * latitude   (latitude) float64 -90.0 -88.11 -86.21 ... 84.32 86.21 88.11
  * longitude  (longitude) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
Data variables:
    s4max      (latitude, longitude) float64 dask.array<chunksize=(95, 144), meta=np.ndarray>
Attributes:
    averaging_period:  Monthly

In [15]:
# Save the month's s4max data into a NetCDF file
output_directory ="./Nc_Files/s4max/" 

if len(desired_months) == 1:
    desired_month_label = desired_months[0]  # Use the first (and only) element
else:
    desired_month_label = "_".join(map(str, desired_months))  # Insert all elements separated by underscores

output_file = f"{output_directory}s4max_{year}_{desired_month_label}.nc"


s4max.to_netcdf('output_file.nc')

KeyboardInterrupt: 

In [30]:
#Sixth attempt - same as 5 but processing day by day
# Computing the daily average variable & Saving to a nc file for each day
# This code *I think* WORKS - sensible looking values are saved in the nc file (though only a few non NaN values in the array I checked)

# LOCAL TIME AND LON

start_time = datetime.now()
print("Start time: ", start_time)


# Manually set the year you want to process
year = 2006  # Set the desired year
desired_months = [12]#, 4]#, 5]  # Set the desired months

year_mask = ds["time"].dt.year == year
year_data = ds.where(year_mask, drop=True)

# Create an empty list to accumulate datasets
daily_lon_datasets = []
daily_LT_dataset1 = []
daily_LT_dataset2 = []

ssn_lon_dataset = []
ssn_LT_dataset1 = []
ssn_LT_dataset2 = []

# Calculate the desired averaging_period based on your logic
if len(desired_months) == 1:
    averaging_period = "Monthly"
elif len(desired_months) == 2:
    averaging_period = "Two-Month"
elif len(desired_months) == 3:
    averaging_period = "Three-Month"
else:
    averaging_period = "Unknown"  # You can specify another value for other cases

output_directory ="./Nc_Files/s4max/"

for month in desired_months:
    month_name = calendar.month_name[month]

    # Create a mask for the current month
    month_mask = year_data["time"].dt.month == month
    month_data = year_data.where(month_mask, drop=True)

    # Month data is filtered, now filter by day
    for day in range(1, calendar.monthrange(year, month)[1] + 1):
        day_mask = month_data["time"].dt.day == day
        day_data = month_data.where(day_mask, drop=True)
        day_time_steps = len(day_data["time"])
        
        s4max_lon = da.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, day_time_steps), np.nan, chunks=(-1, -1, -1, -1))

        s4max_LT = da.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins, day_time_steps), np.nan, chunks=(-1, -1, -1, -1))
        
        # Create a timestamp using these values
        day_timestamp = pd.Timestamp(year=year, month=month, day=day)
        date_string = day_timestamp.strftime('%Y-%m-%d')

        if day_time_steps > 0:
            for i in tqdm(range(day_time_steps), desc=f'Year {year}, Month {month_name}, Day {day}'):
                current_altitude = day_data['altitude_var'].values[i]
                current_lat = day_data['lat'].values[i]
                current_lon = day_data['lon'].values[i]
                current_s4max = day_data['s4max'].values[i]
                current_time = day_data["time"].values[i]
                current_LT = day_data["LT"].values[i]

                # Check if the current_altitude is within the range of your defined bins
                if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                    lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                    lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                    altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                    
                    # Calculate LT_bin_index and manually assign it to the last bin if it's equal to the upper bound
                    LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                    if current_LT == LT_bin_edges[-1]:
                        LT_bin_index = len(LT_bin_edges) - 2

                    # Update the corresponding entry in the s4max data array
                    s4max_lon[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max
                    #s4max_LT[altitude_bin_index, lat_bin_index, LT_bin_index, i] = current_s4max

            #//////////////////////////////////
            # Create a new xarray dataset for the s4max lon data. One day of data at all time steps for that day
            s4max_lon_ds = xr.Dataset(
                data_vars={"s4max": (['altitude', 'latitude', 'longitude', 'time'], s4max_lon)},
                coords={
                    "altitude": Zavg_sl_bin_edges[:-1],
                    "latitude": lat_bin_edges[:-1],
                    "longitude": lon_bin_edges[:-1],
                    "time": day_data["time"].values,
                }
            )
            
            # # Create a new xarray dataset for the s4max LT data. One day of data at all time steps for that day
            # s4max_LT_ds = xr.Dataset(
            #     data_vars={"s4max": (['altitude', 'latitude', 'local_time', 'time'], s4max_LT)},
            #     coords={
            #         "altitude": Zavg_sl_bin_edges[:-1],
            #         "latitude": lat_bin_edges[:-1],
            #         "local_time": LT_bin_edges[:-1],
            #         "time": day_data["time"].values,
            #     }
            # )
            # #//////////////////////////////////
            
            #--------------------------------
            # Average s4max_lon_ds over time dimension
            s4lonavg_alt_lat_lon = s4max_lon_ds['s4max'].mean(dim='time', keep_attrs=True) # daily avg 
            # Average s4lonavg_alt_lat_lon over alt dimension
            s4lonavg_lat_lon = s4lonavg_alt_lat_lon.mean(dim='altitude', keep_attrs=True) # *** daily avg over all alts ***
            # Now create a new DataArray with the desired variable name 's4max'
            s4lonavg_lat_lon = xr.Dataset(
                {
                    's4max': s4lonavg_lat_lon.expand_dims(time=[day_timestamp])
                },
                coords={
                    'latitude': s4lonavg_lat_lon['latitude'],
                    'longitude': s4lonavg_lat_lon['longitude'],
                    'time': [day_timestamp],
                }
            )   
            # #--------------------------------
            # # Average s4max_LT_ds over time dimension 
            # s4LTavg_alt_lat_LT = s4max_LT_ds['s4max'].mean(dim='time', keep_attrs=True) # *** daily avg ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat_LT = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat_LT.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat_LT['latitude'],
            #         'latitude': s4LTavg_alt_lat_LT['latitude'],
            #         'local_time': s4LTavg_alt_lat_LT['local_time'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
            # # Average s4LTavg_alt_lat_LT over LT dimension
            # s4LTavg_alt_lat = s4LTavg_alt_lat_LT.mean(dim='local_time', keep_attrs=True) # *** daily avg over all LTs ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat['altitude'],
            #         'latitude': s4LTavg_alt_lat['latitude'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
        

            with ProgressBar():
                computed_ds = s4lonavg_lat_lon.compute() 
                
            output_file = f"{output_directory}s4lonavg_lat_lon_{date_string}.nc"
            computed_ds.to_netcdf(output_file)
            tqdm.write(f"Saved {output_file}")
            
#             # # Append daily datasets to the list
#             daily_lon_datasets.append(s4lonavg_lat_lon)
#             # daily_LT_dataset1.append(s4LTavg_alt_lat_LT)
#             # daily_LT_dataset2.append(s4LTavg_alt_lat)
#             #--------------------------------
            

# ssn_lon_dataset = xr.concat(daily_lon_datasets, dim='time', data_vars='all')
# # ssn_LT_dataset1 = xr.concat(daily_LT_dataset1, dim='time', data_vars='all')
# # ssn_LT_dataset2 = xr.concat(daily_LT_dataset2, dim='time', data_vars='all')


# ssn_s4lonavg_lat_lon_da = ssn_lon_dataset['s4max'].mean(dim='time', keep_attrs=True)
# # ssn_s4LTavg_alt_lat_LT = ssn_LT_dataset1['s4max'].mean(dim='time', keep_attrs=True)
# # ssn_s4LTavg_alt_lat = ssn_LT_dataset2['s4max'].mean(dim='time', keep_attrs=True)


# # Now create a new DataArray with the desired variable name 's4max'
# ssn_s4lonavg_lat_lon_ds = xr.Dataset(
#     data_vars={"s4max": ssn_s4lonavg_lat_lon_da},
#     attrs={'averaging_period': averaging_period}
# )   



end_time = datetime.now()
time_elapsed = end_time - start_time
print("Time elapsed: ", time_elapsed)


Start time:  2023-10-31 15:11:13.603421


Year 2006, Month December, Day 28: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [00:00<00:00, 3180.76it/s]

[                                        ] | 0% Completed | 166.81 us

[############                            ] | 30% Completed | 56.08 ss


KeyboardInterrupt: 

In [12]:
#Sixth attempt - same as 5 but processing day by day
# Calculating daily average and then dataset (monthly or 3monthly) average. 
# Next it computes the array and saves to a nc file.

#This just crashes the kernel. I think it's due to memory issues. 

# LOCAL TIME AND LON


# Create a Dask cluster using all 80 available cores
cluster = LocalCluster(n_workers=10, threads_per_worker=10)  # You can adjust threads_per_worker as needed

# Create a Dask client to connect to the cluster
client = Client(cluster)


# Manually set the year you want to process
year = 2007  # Set the desired year
desired_months = [3]#, 4]#, 5]  # Set the desired months

year_mask = ds["time"].dt.year == year
year_data = ds.where(year_mask, drop=True)

# Create an empty list to accumulate datasets
daily_lon_datasets = []
daily_LT_dataset1 = []
daily_LT_dataset2 = []

ssn_lon_dataset = []
ssn_LT_dataset1 = []
ssn_LT_dataset2 = []

# Calculate the desired averaging_period based on your logic
if len(desired_months) == 1:
    averaging_period = "Monthly"
elif len(desired_months) == 2:
    averaging_period = "Two-Month"
elif len(desired_months) == 3:
    averaging_period = "Three-Month"
else:
    averaging_period = "Unknown"  # You can specify another value for other cases

output_directory ="./Nc_Files/s4max/"

for month in desired_months:
    month_name = calendar.month_name[month]

    # Create a mask for the current month
    month_mask = year_data["time"].dt.month == month
    month_data = year_data.where(month_mask, drop=True)

    # Month data is filtered, now filter by day
    for day in range(1, calendar.monthrange(year, month)[1] + 1):
        day_mask = month_data["time"].dt.day == day
        day_data = month_data.where(day_mask, drop=True)
        day_time_steps = len(day_data["time"])
        
        s4max_lon = da.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, day_time_steps), np.nan, chunks=(-1, -1, -1, -1))

        #s4max_LT = da.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins, day_time_steps), np.nan, chunks=(-1, -1, -1, -1))
        
        # Create a timestamp using these values
        day_timestamp = pd.Timestamp(year=year, month=month, day=day)
        date_string = day_timestamp.strftime('%Y-%m-%d')

        if day_time_steps > 0:
            for i in tqdm(range(day_time_steps), desc=f'Year {year}, Month {month_name}, Day {day}'):
                current_altitude = day_data['altitude_var'].values[i]
                current_lat = day_data['lat'].values[i]
                current_lon = day_data['lon'].values[i]
                current_s4max = day_data['s4max'].values[i]
                current_time = day_data["time"].values[i]
                current_LT = day_data["LT"].values[i]

                # Check if the current_altitude is within the range of your defined bins
                if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                    lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                    lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                    altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                    
                    # Calculate LT_bin_index and manually assign it to the last bin if it's equal to the upper bound
                    LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                    if current_LT == LT_bin_edges[-1]:
                        LT_bin_index = len(LT_bin_edges) - 2

                    # Update the corresponding entry in the s4max data array
                    s4max_lon[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max
                    #s4max_LT[altitude_bin_index, lat_bin_index, LT_bin_index, i] = current_s4max

            #//////////////////////////////////
            # Create a new xarray dataset for the s4max lon data. One day of data at all time steps for that day
            s4max_lon_ds = xr.Dataset(
                data_vars={"s4max": (['altitude', 'latitude', 'longitude', 'time'], s4max_lon)},
                coords={
                    "altitude": Zavg_sl_bin_edges[:-1],
                    "latitude": lat_bin_edges[:-1],
                    "longitude": lon_bin_edges[:-1],
                    "time": day_data["time"].values,
                }
            )
            
            s4max_lon = None
            
            # # Create a new xarray dataset for the s4max LT data. One day of data at all time steps for that day
            # s4max_LT_ds = xr.Dataset(
            #     data_vars={"s4max": (['altitude', 'latitude', 'local_time', 'time'], s4max_LT)},
            #     coords={
            #         "altitude": Zavg_sl_bin_edges[:-1],
            #         "latitude": lat_bin_edges[:-1],
            #         "local_time": LT_bin_edges[:-1],
            #         "time": day_data["time"].values,
            #     }
            # )
            # #//////////////////////////////////
            
            #--------------------------------
            # Average s4max_lon_ds over time dimension
            s4lonavg_alt_lat_lon = s4max_lon_ds['s4max'].mean(dim='time', keep_attrs=True) # daily avg 
            # Average s4lonavg_alt_lat_lon over alt dimension
            s4lonavg_lat_lon = s4lonavg_alt_lat_lon.mean(dim='altitude', keep_attrs=True) # *** daily avg over all alts ***
            # Now create a new DataArray with the desired variable name 's4max'
            
            s4lonavg_alt_lat_lon = None
            
            s4lonavg_lat_lon = xr.Dataset(
                {
                    's4max': s4lonavg_lat_lon.expand_dims(time=[day_timestamp])
                },
                coords={
                    'latitude': s4lonavg_lat_lon['latitude'],
                    'longitude': s4lonavg_lat_lon['longitude'],
                    'time': [day_timestamp],
                }
            )  
            
            
            
            # #--------------------------------
            # # Average s4max_LT_ds over time dimension 
            # s4LTavg_alt_lat_LT = s4max_LT_ds['s4max'].mean(dim='time', keep_attrs=True) # *** daily avg ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat_LT = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat_LT.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat_LT['latitude'],
            #         'latitude': s4LTavg_alt_lat_LT['latitude'],
            #         'local_time': s4LTavg_alt_lat_LT['local_time'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
            # # Average s4LTavg_alt_lat_LT over LT dimension
            # s4LTavg_alt_lat = s4LTavg_alt_lat_LT.mean(dim='local_time', keep_attrs=True) # *** daily avg over all LTs ***
            # # Now create a new DataArray with the desired variable name 's4max'
            # s4LTavg_alt_lat = xr.Dataset(
            #     {
            #         's4max': s4LTavg_alt_lat.expand_dims(time=[day_timestamp])
            #     },
            #     coords={
            #         'altitude': s4LTavg_alt_lat['altitude'],
            #         'latitude': s4LTavg_alt_lat['latitude'],
            #         'time': [day_timestamp],
            #     }
            # )
            # #--------------------------------
        

#             with ProgressBar():
#                 computed_ds = s4lonavg_lat_lon.compute() 
                
#             output_file = f"{output_directory}s4lonavg_lat_lon_{date_string}.nc"
#             computed_ds.to_netcdf(output_file)
#             tqdm.write(f"Saved {output_file}")
            
            # # Append daily datasets to the list
            daily_lon_datasets.append(s4lonavg_lat_lon)
            # daily_LT_dataset1.append(s4LTavg_alt_lat_LT)
            # daily_LT_dataset2.append(s4LTavg_alt_lat)
            #--------------------------------
            
s4lonavg_lat_lon = None

ssn_lon_dataset = xr.concat(daily_lon_datasets, dim='time', data_vars='all')
# ssn_LT_dataset1 = xr.concat(daily_LT_dataset1, dim='time', data_vars='all')
# ssn_LT_dataset2 = xr.concat(daily_LT_dataset2, dim='time', data_vars='all')

daily_lon_datasets = [] 

ssn_s4lonavg_lat_lon_da = ssn_lon_dataset['s4max'].mean(dim='time', keep_attrs=True)
# ssn_s4LTavg_alt_lat_LT = ssn_LT_dataset1['s4max'].mean(dim='time', keep_attrs=True)
# ssn_s4LTavg_alt_lat = ssn_LT_dataset2['s4max'].mean(dim='time', keep_attrs=True)

ssn_lon_dataset = None

# Now create a new DataArray with the desired variable name 's4max'
ssn_s4lonavg_lat_lon_ds = xr.Dataset(
    data_vars={"s4max": ssn_s4lonavg_lat_lon_da},
    attrs={'averaging_period': averaging_period}
)   

ssn_s4lonavg_lat_lon_da = None



with ProgressBar():
    computed_ds = ssn_s4lonavg_lat_lon_ds.compute() 
    

if len(desired_months) == 1:
    desired_month_label = desired_months[0]  # Use the first (and only) element
else:
    desired_month_label = "_".join(map(str, desired_months))  # Insert all elements separated by underscores

output_file = f"{output_directory}ssn_s4lonavg_lat_lon_ds_{year}_{desired_month_label}.nc"


computed_ds.to_netcdf(output_file)
tqdm.write(f"Saved {output_file}")

client.close()
cluster.close()


/localhome/sestay/miniconda3/envs/myenv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46163 instead
  warnings.warn(
Year 2007, Month March, Day 7:  42%|████████████████████████████████████████████████▍                                                                  | 2666/6334 [00:01<00:02, 1440.72it/s]


KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-10-31 16:31:18,082 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-10-31 16:31:18,082 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
2023-10-31 16:31:18,083 - distributed.nanny - ERROR - Worker process died unexpectedly
  File "/localhome/sestay/miniconda3/envs/myenv/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/localhome/sestay/miniconda3/envs/myenv/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/localhome/sestay/miniconda3/envs/myenv/lib/python3.11/site-packages/distributed/nanny.py", line 955, in run
    await worker.finished()
  File "/localhome/sestay/miniconda3/envs/myenv/lib/python3.11/s

In [11]:
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [ ]:
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * *Working code END * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##
## * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ##

In [ ]:
#################################################################################################################

In [ ]:
############################ OLD CODE ####################################

In [ ]:
#################################################################################################################

In [10]:
# Get available CPU cores
num_cores = os.cpu_count()

# Get available RAM in GB
ram_gb = round(psutil.virtual_memory().total / (1024 ** 3), 2)

num_cores, ram_gb

(80, 125.26)

In [11]:
from dask.distributed import Client

# Initialize the Dask Client
client = Client(n_workers=num_cores, threads_per_worker=1, memory_limit=f"{ram_gb}GB")


In [12]:

# Convert your list to a NumPy array
Zavg_sl_bin_edges = np.array(Zavg_sl_bin_edges)


# Assuming you have 'lat_bin_edges', 'lon_bin_edges', and 'Zavg_sl_bin_edges' as NumPy arrays
lat_bins = da.from_array(lat_bin_edges)
lon_bins = da.from_array(lon_bin_edges)
altitude_bins = da.from_array(Zavg_sl_bin_edges)


In [13]:
num_time_steps = len(ds['nfile'])

# Create an empty Dask array for s4max_data
s4max_data = da.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, num_time_steps), chunks=(-1, -1, -1, -1))


In [14]:

# Create Dask arrays for latitude, longitude, altitude, and s4max
lat_dask = da.from_array(ds_lat)
lon_dask = da.from_array(ds_lon)
altitude_dask = da.from_array(ds_alt)
s4max_dask = da.from_array(ds_s4max)

# Find the bin indices for latitude, longitude, and altitude
lat_bin_indices = da.digitize(lat_dask, lat_bin_edges) - 1
lon_bin_indices = da.digitize(lon_dask, lon_bin_edges) - 1
altitude_bin_indices = da.digitize(altitude_dask, Zavg_sl_bin_edges) - 1

# Update the corresponding entries in the s4max_data Dask array
s4max_data = da.store(
    s4max_data,
    s4max_dask,
    (altitude_bin_indices, lat_bin_indices, lon_bin_indices, da.arange(num_time_steps))
)

# Now s4max_data is a 4D Dask array containing categorized s4max data for all time steps


TypeError: Targets must be either Delayed objects or array-likes

In [ ]:
# Compute the Dask array to obtain the categorized s4max data
s4max_data = s4max_data.compute()

In [ ]:
client.close()

In [ ]:
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################

In [22]:

# Initialize an empty multidimensional array to store categorized s4max data

num_time_steps = len(ds['nfile'])
#print("Number of elements in 'nfile':", timelen)

# Create a 4D array filled with NaN values to store s4max data
s4max_data = np.empty((Zavg_sl_num_bins, lat_num_bins, lon_num_bins, num_time_steps))
s4max_data[:] = np.nan



# Loop through your data to categorize s4max
for i in range(num_time_steps):
    # Extract the data at the current time step
    current_lat = ds['lat_var'][i]
    current_lon = ds['lon_var'][i]
    current_altitude = ds['altitude_var'][i]
    current_s4max = ds['s4max'][i]

    # Find the bin indices for latitude, longitude, and altitude
    lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
    lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
    altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges) - 1

    # Update the corresponding entry in the s4max_data array
    s4max_data[altitude_bin_index, lat_bin_index, lon_bin_index, i] = current_s4max

# Now s4max_data is a 4D array containing categorized s4max data


MemoryError: Unable to allocate 16.0 TiB for an array with shape (25, 95, 144, 6449380) and data type float64

In [ ]:
def categorize_s4max(lat_indices, lon_indices, altitude_indices, s4max_values):
    
    # Your categorization logic here
    result = da.zeros((len(altitude_indices), len(lat_indices), len(lon_indices))
    for i in range(len(lat_indices)):
        for j in range(len(lon_indices)):
            for k in range(len(altitude_indices)):
                if (
                    altitude_indices[k] >= 0 and altitude_indices[k] < Zavg_sl_num_bins and
                    lat_indices[i] >= 0 and lat_indices[i] < lat_num_bins and
                    lon_indices[j] >= 0 and lon_indices[j] < lon_num_bins
                ):
                    result[k, i, j] = s4max_values[i, j, k]
                      
    return result

                      
# Assuming you have 's4max_values' as a Dask array containing s4max data
s4max_data = da.map_blocks(
    categorize_s4max, lat_bins, lon_bins, altitude_bins, s4max_values, dtype=float  # Adjust dtype as needed
)